In [ ]:
%%capture
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 0
%config SqlMagic.displaycon = False

In [ ]:
import datetime

start = datetime.datetime.now() - datetime.timedelta(days=90)
end = datetime.datetime.now()

In [ ]:
%%sql report_districts <<
SELECT 
    r.district as district,
    r.primary_statute as classification,
    r.primary_complaint as complaint,
    r.location_state as state,
    COUNT(*) count
FROM cts_forms_report r
WHERE district notnull AND primary_complaint notnull
GROUP BY complaint, district, classification, state
HAVING COUNT(*) > 1
ORDER BY count desc

In [ ]:
from IPython.display import display, HTML
import numpy as np
primary_complaint_choices = {
    'workplace': 'Workplace discrimination or other employment-related problem',
    'housing': 'Housing discrimination or harassment',
    'education': 'Discrimination at a school, educational program or service, or related to receiving education',
    'police': 'Mistreated by police, correctional staff, or inmates',
    'voting': 'Voting rights or ability to vote affected',
    'commercial_or_public': 'Discriminated against in a commercial location or public place',
    'something_else': 'Something else happened',
}
readable_start = start.strftime("%B %d, %Y")
readable_end = end.strftime("%B %d, %Y")
report_district_data = report_districts.DataFrame()
districts = report_district_data['district'].values
data_divs = []
district_dropdown = []
for i, district in enumerate(districts):
    district_dropdown.append(
        f'<option value="{district}">{district}</option>'
    )
    complaint_choice = report_district_data['complaint'].values[i]
    complaint = primary_complaint_choices.get(complaint_choice)
    classification = report_district_data['classification'].values[i]
    state = report_district_data['state'].values[i]
    count = report_district_data['count'].values[i]
    data_divs.append(
        f"""<div class="flex-align-center district-wrapper {district} flex-row grid-row flex-justify display-flex border-bottom border-base-light padding-top-1 padding-bottom-1">
            <span class="grid-col-3">{complaint}</span>
            <span class="grid-col-1">{district}</span>
            <span class="grid-col-1">{state}</span>
            <span class="grid-col-1">{classification}</span>
            <a class="grid-col-2 flex-align-self-end" href="http://localhost:8000/form/view/?district={district}&primary_complaint={complaint}&primary_statute={classification}&grouping=default">
                <strong>View {count} reports</strong>
            </a>
        </div>
    """)

display(HTML(f"""
    <div class="crt-portal-card light maxh-mobile-lg overflow-y-scroll">
    <h3>Repeated primary complaints by district</h3>
    <select name="incident_district" class="usa-select incident-district" aria-label="Incident district">
        <option value="">Select a district</option>
        {''.join(sorted(np.unique(district_dropdown)))}
    </select>
    <p style="font-size: 0.75rem">
      <em>Repeated report types by primary complaint and district submitted between
        <strong>{readable_start}</strong> and
        <strong>{readable_end}</strong>
      </em>
    </p>
    <br />
    <div class="flex-align-center districts flex-row grid-row flex-justify display-flex">
        <strong class="grid-col-3">Primary complaint</strong>
        <strong class="grid-col-1">District</strong>
        <strong class="grid-col-1">State</strong>
        <strong class="grid-col-1">Classification</strong>
        <strong class="grid-col-2 flex-align-self-end"># of Reports</strong>
    </div>
    <hr />
    {''.join(data_divs)}
    </div>
"""))
